In [9]:
# 1. 導入必要的套件
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper
from langchain.tools import Tool
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents import create_react_agent
from langchain import hub

# 2. 設置環境
import os
from dotenv import load_dotenv
load_dotenv()

# 3. 初始化 LLM
llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.1)

# 4. 設置搜尋工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="""
        用於搜尋資訊的工具。
        """
    ),
]

# 5. 設置記憶存儲
memory = ConversationTokenBufferMemory(
    memory_key="chat_history", # 在 prompt 中使用 {chat_history} 來引用歷史對話
    return_messages=True, # 返回格式為消息列表而不是字符串
    output_key="output", # 指定要存儲在歷史記錄中的輸出名，預設為"output" 其實可以不用打
    max_token_limit=4000,  # 記憶體限制，超過限制時，會自動刪除最舊的對話，預設2000
    llm=llm
)

# 6. 創建整合的 prompt 模板
from langchain.prompts import PromptTemplate

template = """你是一個專業的投資顧問助手。使用以下工具來回答人類的問題。請使用繁體中文回答。

{tools}

必須遵循以下規則：
1. 回答格式必須是：
Question: 當前問題
Thought: 分析目前掌握的資訊，並規劃下一步
Action: {tool_names} 中的一個工具名稱
Action Input: 工具的輸入
Observation: 工具的結果
... (重複上述 Thought/Action/Action Input/Observation 直到蒐集足夠資訊)
Thought: 總結所有資訊
Final Answer: 完整的繁體中文回答

2. 每個 Action 必須是上述工具之一

3. 回答必須參考對話歷史

對話歷史：
{chat_history}

Question: {input}
{agent_scratchpad}"""

# 創建 prompt 模板
prompt = PromptTemplate(
    template=template,
    input_variables=["input", "chat_history", "agent_scratchpad"], # 動態提供的參數，問答的記憶，用戶的input都是會變動的
    partial_variables={"tools": tools, "tool_names": ", ".join([tool.name for tool in tools])} # 靜態變數，工具是事前定義好的，回答過程中不會變動
)

# 創建 ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools, # 告訴 AI "你可以使用這些工具"
    prompt=prompt
)

# 創建 agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,# 實際執行工具的地方
    verbose=True,
    memory=memory,
    max_iterations=5,
    early_stopping_method="force",
    handle_parsing_errors=True
)

# 8. 測試對話
def chat_with_agent(question):
    try:
        response = agent_executor.invoke({
            "input": question,
        })
        
        answer = response.get("output", "")
        
        # 對投資相關問題進行補充
        if any(keyword in question for keyword in ["投資", "買進", "賣出", "值得", "前景", "分析"]):
            if len(answer) < 200:  # 如果回答太短，進行追加搜索
                additional_search = search.run(f"{question} 更多資訊 分析")
                answer += f"\n\n補充資訊：\n{additional_search}"
            
            if "風險" not in answer:
                answer += "\n\n投資風險提醒：\n1. 以上分析僅供參考\n2. 投資前請審慎評估風險\n3. 建議諮詢專業投資顧問"
        
        return answer
        
    except Exception as e:
        return f"處理問題時發生錯誤：{str(e)}\n建議重新提問或提供更具體的資訊。"

In [6]:
# 9. 執行測試
question1 = "鈊象電子2024的營收狀況如何？"
print("[問題1]:", question1)
print("[AI回答]:", chat_with_agent(question1))

question2 = "以投資的角度來看，前面的資訊是否足夠，若不足夠，請補充"
print("\n[問題2]:", question2)
print("[AI回答]:", chat_with_agent(question2))

question3 = "根據前面的資訊，你覺得這家公司值得投資嗎？"
print("\n[問題3]:", question3)
print("[AI回答]:", chat_with_agent(question3))

問題1: 鈊象電子2024的營收狀況如何？


> Entering new AgentExecutor chain...
Thought: 為了回答這個問題，我需要先找到關於鈊象電子2024年的營收預測或者是分析師的看法。這將涉及尋找最新的財經報導、分析師報告或者是公司自己發布的未來展望聲明。

Action: Search
Action Input: 鈊象電子 2024 營收預測['鈊象12月營收站上17億大關，在2024年最後一個月創下年營收和月營收的歷史新高。 12月營收年增29.1%，累積去年營收185.12億，年增30.6%，再次創下歷史新高。', '台灣遊戲開發商鈊象（3293），11/6日公布2024年10月自結合併營收，為16.8億元，相較上個月15.81億元，月增6.23%，相對去年同期12.31億元，年增36.48%，月營收也 ...', '鈊象2024年第四季營收50.61億元，亦創單季新高，較前一年度成長33.5％。 累計2024年全年度鈊象電子的營收為185.12億元，年增30.59％，亦為歷史新高。 整 ...', '三大法人上周買超358張。 從基本面來看，鈊象電子2024年前11月已繳出亮眼成績單，前11月累計營收為168億元，年增30.75％。 ... 預測，不必然代表投資之績效。', '法人認為，隨著農曆年旺季將至，估計鈊象今年業績仍有望持續走升，加上海外線上博弈執照將陸續到手，有助整體營收規模成長。 展望未來，鈊象目前正積極申請英國 ...', '法人認為未來2年，每年能成長21%，不過要注意的是這只有2年的預估。 最後再與今年累計營收成長率相比，這個方法在營收趨勢穩定的公司比較適用，鈊象剛好 ...', '鈊象12月營收站上17億大關，在2024年最後一個月創下月營收的歷史新高。 12月營收年增29.1%，累積去年營收185.12億，年增30.6%，再次創下歷史新高。鈊象從2018年的營收30億 ...', '鈊象(3293-TW)今天公告2024年10月營收為新台幣16.80億元，年增率36.48%，月增率6.24%。 今年1-10月累計營收為151.31億元，累計年增率30.25%。', '... 收入來源。2023年，美國授權收入佔整體營收的15%。 整體成長預期. 展望2024年，法人平均預估鈊象2024年營收年增率